# Metagraph and Metapath Generation

Here, we use the same syntax as [HetNetPy](https://github.com/hetio/hetnetpy/tree/e1ff1f8296b766dd5a63e5870a8a1a7d82427a80) to create a metagraph and extract metapaths from the KG.

Metagraph creation is based on [this notebook](https://github.com/dhimmel/integrate/blob/93feba1765fbcd76fd79e22f25121f5399629148/integrate.ipynb).

In [1]:
import pandas as pd
import seaborn as sns
import os
import os.path as osp
import numpy as np

import hetnetpy.hetnet
import hetnetpy.readwrite
import hetnetpy.stats

from hetnetpy.abbreviation import metaedges_from_metapath
from hetnetpy.pathtools import DWPC, paths_between

import networkx as nx
from networkx import DiGraph, MultiDiGraph, connected_components

import re
import json
from tqdm import tqdm
from collections import Counter

from itertools import chain

%matplotlib inline

In [2]:
KG_DATA_PATH = '../data/kg/splits/'

MOA_NET = os.path.join(KG_DATA_PATH, 'MoA-net')

MOA_NET_10K = os.path.join(MOA_NET, '10k')

PROT_MOA_NET = os.path.join(KG_DATA_PATH, 'MoA-net-protclass')

Let's load in MoA-net-10k, the version of MoA-net which was cut down by the automatic KG trimming feature:

In [3]:
def create_graph_from_nxobj(nx_graph_obj, node_mapping_file, edge_mapping_file):
    graph = DiGraph()

    old_graph = nx.read_graphml(nx_graph_obj,
                    node_type=int,
                    edge_key_type=int,
                    force_multigraph=True)
    
    for u, v, data in old_graph.edges(data=True):
        graph.add_edge(
            node_mapping_file[u],
            node_mapping_file[v],
            polarity=edge_mapping_file[data['type']],
        )

    connected_components_subgraph = [
        component
        for component in sorted(
            connected_components(
                graph.to_undirected()
            ),
            key=len,
            reverse=True
        )
    ]

    final_subgraph = graph.subgraph(connected_components_subgraph[0])

    return final_subgraph

In [4]:
# read the entity mapper in
entity_mapping = json.load(open(os.path.join(MOA_NET, 'MARS/vocab/entity_vocab.json')))
entity_mapping = {int(v): k for k, v in entity_mapping.items()}
relation_mapping = json.load(open(os.path.join(MOA_NET, 'MARS/vocab/relation_vocab.json')))
relation_mapping = {int(v): k for k, v in relation_mapping.items()}

In [5]:
kg = create_graph_from_nxobj(os.path.join(MOA_NET_10K, 'nx_graph.graphml'),
                                 entity_mapping,
                                 relation_mapping)

In [6]:
test_10k = np.load(os.path.join(MOA_NET_10K, 'test-10k.npy'))
val_10k = np.load(os.path.join(MOA_NET_10K, 'dev-10k.npy'))

Drop those from the test set which are no longer connected:

In [7]:
gold_standard = {'source': [], 'target': []}
test_10k_df = {'source': [], 'target': [], 'relation': []}

for source, rel, target in test_10k:
    e1 = entity_mapping[source]
    e2 = entity_mapping[target]
    r = relation_mapping[rel]

    if e1 in kg and e2 in kg and nx.has_path(kg, e1, e2) and nx.shortest_path_length(kg, e1, e2) <= 4:
        gold_standard['source'].append(e1)
        test_10k_df['source'].append(e1)
        gold_standard['target'].append(e2)
        test_10k_df['target'].append(e2)
        test_10k_df['relation'].append(r)

gold_standard = pd.DataFrame(gold_standard)
test_10k_df = pd.DataFrame(test_10k_df)
print(len(gold_standard))
print(len(test_10k_df))

100
100


In [8]:
validation_set = {'source': [], 'target': [], 'relation': []}

for source, rel, target in val_10k:
    e1 = entity_mapping[source]
    e2 = entity_mapping[target]
    r = relation_mapping[rel]

    if e1 in kg and e2 in kg and nx.has_path(kg, e1, e2) and nx.shortest_path_length(kg, e1, e2) <= 4:
        validation_set['source'].append(e1)
        validation_set['target'].append(e2)
        validation_set['relation'].append(r)

validation_set = pd.DataFrame(validation_set)
print(len(validation_set))

90


### Define and Instantiate Metagraph:

In [9]:
kind_to_abbev = {
    
    # metanodes
    'Compound': 'C',
    'Gene': 'G',
    'Biological Process': 'BP',
    
    # metaedges
    'CuG': 'u',
    'CdG': 'd',
    'GiG': 'i',
    'GpBP': 'p',
    'CtBP': 't',
}

metaedge_tuples = [
    ('Compound', 'Gene', 'CuG', 'forward'),
    ('Compound', 'Gene', 'CdG', 'forward'),
    ('Gene', 'Gene', 'GiG', 'forward'),
    ('Gene', 'Biological Process', 'GpBP', 'forward'),
    ('Compound', 'Biological Process', 'CtBP', 'forward')
]

metagraph = hetnetpy.hetnet.MetaGraph.from_edge_tuples(metaedge_tuples, kind_to_abbev)
graph = hetnetpy.hetnet.Graph(metagraph)

In [10]:
def map2type(node_name):
    if 'ncbigene' in node_name:
        return 'Gene'
    elif 'GO:' in node_name:
        return 'Biological Process'
    elif 'pubchem' in node_name:
        return 'Compound'

In [11]:
seen = set()
all_bps = set()

for u, v, data in kg.edges(data=True):
    if '_' in data['polarity']:
        continue
    if not u in seen:
        graph.add_node(kind=map2type(u), identifier=u)
        seen.add(u)
        if map2type(u) == 'Biological Process':
            all_bps.add(u)
    if not v in seen:
        graph.add_node(kind=map2type(v), identifier=v)
        seen.add(v)
        if map2type(v) == 'Biological Process':
            all_bps.add(v)

    src_id = map2type(u), u
    trgt_id = map2type(v), v

    graph.add_edge(source_id=src_id, target_id=trgt_id, kind=data['polarity'], direction='forward')

## Metapath Extraction

Here, we have functions which can extract metapaths:

Extract all metapaths up to a certain length:

Extract metapaths from a given source node type (and, optionally, to a target node type):

In [12]:
tgt_metapaths = metagraph.extract_metapaths(source='Compound', target='Biological Process')
print(tgt_metapaths)

[Ct>BP, Cd>Gp>BP, Cu>Gp>BP, Ct>BP<tCt>BP, Ct>BP<pGp>BP, Cd>G<dCt>BP, Cd>G<uCt>BP, Cd>G<iGp>BP, Cd>Gi>Gp>BP, Cu>G<dCt>BP, Cu>G<uCt>BP, Cu>G<iGp>BP, Cu>Gi>Gp>BP, Ct>BP<tCd>Gp>BP, Ct>BP<tCu>Gp>BP, Ct>BP<pG<dCt>BP, Ct>BP<pG<uCt>BP, Ct>BP<pG<iGp>BP, Ct>BP<pGi>Gp>BP, Cd>Gp>BP<tCt>BP, Cd>Gp>BP<pGp>BP, Cd>G<dCd>Gp>BP, Cd>G<dCu>Gp>BP, Cd>G<uCd>Gp>BP, Cd>G<uCu>Gp>BP, Cd>G<iG<dCt>BP, Cd>G<iG<uCt>BP, Cd>G<iG<iGp>BP, Cd>G<iGi>Gp>BP, Cd>Gi>G<dCt>BP, Cd>Gi>G<uCt>BP, Cd>Gi>G<iGp>BP, Cd>Gi>Gi>Gp>BP, Cu>Gp>BP<tCt>BP, Cu>Gp>BP<pGp>BP, Cu>G<dCd>Gp>BP, Cu>G<dCu>Gp>BP, Cu>G<uCd>Gp>BP, Cu>G<uCu>Gp>BP, Cu>G<iG<dCt>BP, Cu>G<iG<uCt>BP, Cu>G<iG<iGp>BP, Cu>G<iGi>Gp>BP, Cu>Gi>G<dCt>BP, Cu>Gi>G<uCt>BP, Cu>Gi>G<iGp>BP, Cu>Gi>Gi>Gp>BP]


We do not want any with a compound in the middle:

In [13]:
CtoBP_metapaths = []


for i in tgt_metapaths:
    if (repr(i) != 'Ct>BP') & (Counter(re.split('>|<|[^A-Z]', repr(i)))['C'] == 1):
        CtoBP_metapaths.append(i)

How many did it prune?

In [14]:
print(len(CtoBP_metapaths))
print(len(tgt_metapaths))

19
47


## Metrics based on Metapaths

First, we need the KG with the test BPs:

In [15]:
def get_paths(
    graph,
    source,
    target,
    rule,
):
    try:
        paths = paths_between(
            graph,
            source=('Compound', source),
            target=('Biological Process', target),
            metapath=tuple(rule),
            duplicates=False,
        )
    except Exception as e:
        print(e)
        paths = []        
    
    return paths

In [17]:
hits_at_10 = []
hits_at_3 = []
hits_at_1 = []
mrr = []

for i, row in tqdm(test_10k_df.iterrows(), total=len(test_10k_df)):
    
    source = row['source']
    target = row['target']

    rank = {}
    
    for bp in all_bps:
        
        all_paths = []
    
        # All metapaths going from compound to biological process
        for rule in CtoBP_metapaths:
                    
            paths = get_paths(
                graph=graph,
                source=source,
                target=bp,
                rule=rule,
            )
            
            if not paths:
                continue
                
            all_paths.append(paths)
                    
        # flatten paths
        all_paths = list(chain(*all_paths))
        
        if not all_paths:
            continue
    
        score = DWPC(all_paths, damping_exponent=0.4)
                
        rank[bp] = score
        
    # Sort rank by score
    ranked_bp_predictions = [
        go
        for go, _ in sorted(rank.items(), key=lambda x: x[1], reverse=True)
    ]
            
    # Hits at 10
    if target in ranked_bp_predictions[:10]:
        hits_at_10.append(1) 
    else:
        hits_at_10.append(0)
    
    # Hits at 3
    if target in ranked_bp_predictions[:3]:
        hits_at_3.append(1)
    else:
        hits_at_3.append(0)
    
    # Hits at 1
    if target in ranked_bp_predictions[:1]:
        hits_at_1.append(1)
    else:
        hits_at_1.append(0)
        
    # MRR
    if target in ranked_bp_predictions:
        print(f"{ranked_bp_predictions.index(target)} / {len(ranked_bp_predictions)}")
        mrr.append(
            1.0 / (ranked_bp_predictions.index(target) + 1)
        )
    else:
        mrr.append(0)

        
print(f'Hits at 10: {sum(hits_at_10) / len(hits_at_10)}')
print(f'Hits at 3: {sum(hits_at_3) / len(hits_at_3)}')
print(f'Hits at 1: {sum(hits_at_1) / len(hits_at_1)}')
print(f'MRR: {sum(mrr) / len(mrr)}')

  1%|          | 1/100 [00:37<1:01:04, 37.02s/it]

20 / 79


  2%|▏         | 2/100 [00:38<26:12, 16.05s/it]  

1 / 42


  3%|▎         | 3/100 [00:41<16:11, 10.01s/it]

...

21 / 62


 99%|█████████▉| 99/100 [19:00<00:11, 11.79s/it]

7 / 73


100%|██████████| 100/100 [19:24<00:00, 11.64s/it]

8 / 78
Hits at 10: 0.78
Hits at 3: 0.56
Hits at 1: 0.37
MRR: 0.5077486538860714
